# Ensemble & Submit

## Imports

In [ ]:
from keras.optimizers import Adam
from keras.layers.advanced_activations import PReLU
import numpy as np
import matplotlib.pyplot as plt
from optimizers.AdamAccumulate import AdamAccumulate
from models.u_net import UNet
from models.tiramisu import Tiramisu
from submit_ensemble import generate_submit_ensemble
from submit_ensemble import ModelInfo
import utils

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Create & Load Models

In [ ]:
models_info = []

model0 = UNet((1024, 1024, 3), filters=10, depth=4, activation=lambda x: PReLU()(x))
model0.compile(Adam(), loss='binary_crossentropy')
model0.load_weights('weights/unet-2017-09-01-1355.hdf5')
models_info.append(ModelInfo(lambda batch: model0.predict_on_batch(batch), 1024, True, 1))

model1 = UNet((1024, 1024, 3), filters=8, depth=7, activation=lambda x: PReLU()(x))
model1.compile(Adam(), loss='binary_crossentropy')
model1.load_weights('weights/unet-2017-09-02-1809.hdf5')
models_info.append(ModelInfo(lambda batch: model1.predict_on_batch(batch), 1024, False, 1))

model2 = UNet((1024, 1024, 3), filters=8, depth=7, activation=lambda x: PReLU()(x))
model2.compile(Adam(), loss='binary_crossentropy')
model2.load_weights('weights/unet-2017-09-03-1739.hdf5')
models_info.append(ModelInfo(lambda batch: model2.predict_on_batch(batch), 1024, True, 1))

## Load BBoxes

In [ ]:
bbox_file_path = 'inputs/test_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

## Generate Submit

In [ ]:
batch_size = 16
threshold = 0.5
augmentations = 4
submit_name = 'submit-2017-09-10-1125'
test_path = 'inputs/test/'
test_masks_path = 'outputs/test_masks/' #None
generate_submit_ensemble(models_info, batch_size, threshold, test_path, 'outputs/',
                         submit_name, augmentations, test_masks_path, bboxes)

### Visualization

In [ ]:
utils.show_test_masks(test_path, test_masks_path)